In [1]:
!pip install langchain qdrant-client -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# **Xử lý dữ liệu**

In [6]:
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader,TextLoader
loader_datafptu = TextLoader("/content/datasetfptu.txt")

In [7]:
documents_datafptu = loader_datafptu.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800,
                                                chunk_overlap=400)

texts_datafptu = text_splitter.split_documents(documents_datafptu)


In [8]:
len(texts_datafptu)

44

In [9]:
# Cài đặt thông tin truy cập vector database
QDRANT_API_KEY = "<...>"
QDRANT_URL = "<...>"
HUGGINGFACE_API_KEY= "<...>"

In [11]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
# Tạo lớp embeddings
embeddings = HuggingFaceInferenceAPIEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    api_key = HUGGINGFACE_API_KEY,
    model_kwargs = {'device': 'auto'}
)

# Encode dữ liệu text qua lớp embedding,
# rồi lưu lại trên vector database server
qdrant = Qdrant.from_documents(
    texts_datafptu,
    embeddings,
    url=QDRANT_URL,
    prefer_grpc=False,
    collection_name="nttu_sotay_vector_db_v1",
    api_key=QDRANT_API_KEY,
)

print("Tạo vector database thành công!")

Tạo vector database thành công!


In [12]:
# Test tìm văn bản liên quan đến query
query = "Cấu trúc đề thi học bổng Đại học FPT"
found_docs = qdrant.similarity_search(query,k=15)

In [13]:
found_docs

[Document(page_content='Trường Đại học FPT trân trọng thông báo chương trình học bổng “FPTU Scholarship 1000” hệ đại học chính quy – Trường Đại học FPT năm 2023 như sau:\n\n\n1. Nội dung chương trình\nTrao 1000 học bổng hệ đại học chính quy – Trường ĐH FPT năm 2023 cho thí sinh tốt nghiệp THPT năm 2023 đạt các tiêu chí:\n- Đủ điều kiện nhập học vào Trường ĐH FPT (Theo phương thức tuyển sinh:  xét tuyển thẳng hoặc xét kết quả xếp hạng học sinh THPT năm 2023 - chứng nhận thực hiện trên trang https://SchoolRank.fpt.edu.vn – xem trong Quy chế tuyển sinh Hệ đại học chính quy năm 2023 của Trường ĐH FPT);\n- Nộp đơn và được Hội đồng Học bổng Trường ĐH FPT xem xét chấp nhận;\n- Hoàn tất các thủ tục đăng ký học bổng và đăng ký xét tuyển sau khi được chấp nhận cấp học bổng.', metadata={'source': '/content/datasetfptu.txt', '_id': '540840f3-9c78-4034-8419-011235bc88d9', '_collection_name': 'nttu_sotay_vector_db_v1'}),
 Document(page_content='Các bạn thí s